In [ ]:
!pip install openai

# Fine-Tuning and model preparation

In [ ]:
!openai tools fine_tunes.prepare_data -f sample_data/non_patronizing.jsonl

Analyzing...

- Your file contains 9476 prompt-completion pairs

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "sample_data/non_patronizing.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `` for the model to start generating completions, rather than continuing with the prompt.
Once your model starts training, it'll approximately take 2.21 hours to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [ ]:
!openai -k 'API-KEY' api fine_tunes.create -t "PATH-TO-JSONL-FILE" -m 'curie' --batch_size 1

Upload progress: 100% 2.82M/2.82M [00:00<00:00, 5.20Git/s]
Uploaded file from sample_data/non_patronizing.jsonl: file-XiBNrC0GQNB3tA97sWDAwVmT
Created fine-tune: ft-xGNWh0bmKrXIHvrGXjCBoFyI
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-01-28 15:34:26] Created fine-tune: ft-xGNWh0bmKrXIHvrGXjCBoFyI
[2022-01-28 15:34:36] Fine-tune costs $6.23
[2022-01-28 15:34:37] Fine-tune enqueued. Queue number: 0
[2022-01-28 15:34:39] Fine-tune started

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-xGNWh0bmKrXIHvrGXjCBoFyI



In [ ]:
!openai -k 'API-KEY' api fine_tunes.follow -i ft-xGNWh0bmKrXIHvrGXjCBoFyI

[2022-01-28 15:34:26] Created fine-tune: ft-xGNWh0bmKrXIHvrGXjCBoFyI
[2022-01-28 15:34:36] Fine-tune costs $6.23
[2022-01-28 15:34:37] Fine-tune enqueued. Queue number: 0
[2022-01-28 15:34:39] Fine-tune started
[2022-01-28 16:29:11] Completed epoch 1/4
[2022-01-28 17:22:56] Completed epoch 2/4
[2022-01-28 19:13:26] Completed epoch 4/4
[2022-01-28 19:13:57] Uploaded model: curie:ft-personal-2022-01-28-19-13-55
[2022-01-28 19:14:00] Uploaded result file: file-io0nDG0tybLHiMgRwpJ9ukqN
[2022-01-28 19:14:01] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-01-28-19-13-55 -p <YOUR_PROMPT>


swap the fine-tune ("ft-xxxx") with your own model

In [ ]:
!openai -k 'API-KEY' api fine_tunes.results -i "ft-dUZNJyh6Ytievn7S7XWxTF1u"

step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy
1,113,1,2.679702073264967,0.0,0.3904761904761905
2,250,2,2.6453868742032416,0.0,0.49242424242424243
3,291,3,3.1884744100272657,0.0,0.2571428571428571
4,324,4,4.218730217078701,0.0,0.35714285714285715
5,405,5,3.0133875971379895,0.0,0.4230769230769231
6,430,6,4.315187041958173,0.0,0.16666666666666666
7,479,7,3.1452425274377065,0.0,0.32608695652173914
8,528,8,3.22789233426253,0.0,0.2558139534883721
9,649,9,2.501062276203205,0.0,0.5221238938053098
10,666,10,3.7824042793363333,0.0,0.2222222222222222
11,707,11,2.582087895530276,0.0,0.3235294117647059
12,772,12,3.699573823279934,0.0,0.296875
13,861,13,2.6171506010624315,0.0,0.39285714285714285
14,958,14,3.1156550028633014,0.0,0.31521739130434784
15,1015,15,3.088697391836571,0.0,0.4117647058823529
16,1080,16,3.4342321971125784,0.0,0.21666666666666667
17,1129,17,2.7344473403306133,0.0,0.5217391304347826
18,1170,18,3.2954768120311204,0.0,0.27777

In [ ]:
!openai -k 'API-KEY' api completions.create -m 'ft-smPfFKRgeuXb0oMxlM9vPHGZ' -p ''

Error: That model does not exist (HTTP status code: 404)


the -p '' in the end gives the fine-tune an empty prompt. Write something else in there if you want to play around. :)

Alright, so now we have a working fine-tuned model! Let's use it..

In [ ]:
import os
import openai

openai.api_key = "API-KEY"
#swap the fine-tune's ID for yours
retrieved_model = openai.FineTune.retrieve(id="ft-smPfFKRgeuXb0oMxlM9vPHGZ")
#swap the model with yours
curie_response = openai.Completion.create(
  model="curie:ft-personal-2022-01-17-10-47-20",
  prompt="",
  n=5,
  max_tokens=54
)



In [ ]:
import pandas as pd

results_curie = [i['text'] for i in curie_response['choices']]

df_pat_curie = pd.DataFrame(results_curie)
df_pat_curie


,0
0,IBM is donating computer power to the non-prof...
1,2015 was an eventful year for disabled footbal...
2,Documentary gives a look into life of the disa...
3,elizabeth charles meets the homeless families ...
4,predicting st johns coating debris removal suc...


Save it as .csv

In [ ]:
compression_opts = dict(method='zip', archive_name='out.csv')  
df_pat_curie.to_csv('patronizing_curie.zip', index=False, compression=compression_opts)  

In [ ]:
!pwd

/content


---
#How about DaVinci?
> this is just for fun, as we'll use Curie







In [ ]:
retrieved_model = openai.FineTune.retrieve(id="ft-dUZNJyh6Ytievn7S7XWxTF1u")
#note: this is our DaVinci model!
davinci_response = openai.Completion.create(
  model="davinci:ft-personal-2022-01-17-11-25-34",
  prompt="",
  temperature=0.9,
  n=5,
  max_tokens= 100
)


NameError: ignored

But be careful here, depending on the value of n this can get rather expensive really quick. 10.000 f.ex should be around 120€.

In [ ]:
import pandas as pd

results_davinci = [i['text'] for i in davinci_response['choices']]

df_pat_davinci = pd.DataFrame(results_davinci)
df_pat_davinci

,0
0,There are endless reports of people making hug...
1,"We want to get out and about in the , helping ..."
2,Maybe it is because seth dunleavy is a husband...
3,seaweed is one of the most useful micro-algae ...
4,"The performance of the late rafael carino , wh..."


In [ ]:
compression_opts = dict(method='zip', archive_name='out.csv')  
df_pat_davinci.to_csv('patronizing_davinci_test.zip', index=False, compression=compression_opts)  

---
Hmm okay - training 10 examples of synthetic data for Curie costs me approx. 0.012 ct
Training DaVinci costs 0.12 ct

Counteracting imbalances in our datasets of Patronizing: ~1000 entries, Non-patronizing: ~9500 entries would cost

Curie: 12€ (Patronizing, 10.000 examples)


---

# Let's sort some stuff out first..

Before we can start generating data, we'll first need to find the mean token lengths of the patronizing and non-patronizing datasets. GPT-3 is currently not able to stop 'earlier' than the set max_tokens, so we do need to pay attention to this.

In [ ]:
import transformers

#That was done using HuggingFace's 'Transformers' library - I'll paste the code here once I've polished it up

 ---

# So - We'll use Curie! So let's get started!

In [ ]:
import os
import openai

openai.api_key = "API-KEY"
#swap the fine-tune's ID for yours
retrieved_model = openai.FineTune.retrieve(id="ft-smPfFKRgeuXb0oMxlM9vPHGZ")
#swap the model with yours
curie_response = openai.Completion.create(
  model="curie:ft-personal-2022-01-17-10-47-20",
  prompt="",
  n=128,
  #mean token length in the 'patronizing' subset was 60.194361, max_tokens was therefore set to 60.
  max_tokens=60
)



KeyboardInterrupt: ignored

And to save it..

In [ ]:
import pandas as pd

results_curie = [i['text'] for i in curie_response['choices']]

df_pat_curie = pd.DataFrame(results_curie)
df_pat_curie.to_csv('patronizing_curie_X.csv', index=False)

df_pat_curie

And for Non-patronizing data..

In [ ]:
  import os
import openai

openai.api_key = "API-KEY"
#swap the fine-tune's ID for yours
retrieved_model = openai.FineTune.retrieve(id="ft-smPfFKRgeuXb0oMxlM9vPHGZ")
#swap the model with yours
curie_response = openai.Completion.create(
  model="curie:ft-personal-2022-01-28-19-13-55",
  prompt="",
  n=5,
  #mean token length in the 'non-patronizing' subset was 54.44217, max_tokens was therefore set to 54.
  max_tokens=54
)


In [ ]:
curie_response

<OpenAIObject text_completion id=cmpl-4VRC7OzDWMUL3qlbA2tzC3XBIhb4l at 0x7fc0645f19b0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " clearlySERIOUS -- Controversial political figure status updates his page on seen some of the discuss on these here parts of the ' thought police ' and ' social credit ' data bracelets these days ... balance on social good or hit with g u Middle Class tax when backlash"
    },
    {
      "finish_reason": "length",
      "index": 1,
      "logprobs": null,
      "text": "reese witherspoon 's legal moonshine operation is barreling along at a breakneck pace , what with all of the recent everybodysize capable expansion . the operation now has apologies to make not only to wife r homelessness and here are a few more"
    },
    {
      "finish_reason": "length",
      "index": 2,
      "logprobs": null,
      "text": "onabasanj substitutes story of an immigrant with own per specs who nev

# Checkup on our different fine-tuned models

---



In [ ]:
import os
import openai

openai.api_key="API-KEY"
openai.FineTune.list()

<OpenAIObject list at 0x7fc0644f6dd0> JSON: {
  "data": [
    {
      "created_at": 1639058117,
      "fine_tuned_model": null,
      "hyperparams": {
        "batch_size": null,
        "learning_rate_multiplier": null,
        "n_epochs": 4,
        "prompt_loss_weight": 0.1,
        "use_packing": null
      },
      "id": "ft-qDhm3RsLrLZhjzeSIJnhks4L",
      "model": "ada",
      "object": "fine-tune",
      "organization_id": "org-ieOdFmCAdgpIZSBhD6qdrY0v",
      "result_files": [],
      "status": "failed",
      "training_files": [
        {
          "bytes": 319368,
          "created_at": 1639058116,
          "filename": "patronizing.jsonl",
          "id": "file-MciiJxaqRvm9F48oBDYb8Hzl",
          "object": "file",
          "purpose": "fine-tune",
          "status": "error",
          "status_details": "Invalid file format. Example 994 cannot be parsed. Error: line contains invalid json: Expecting value: line 2 column 1 (char 1) (line 994)"
        }
      ],
      "upda